Neural Prophet

In [98]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet, set_log_level
import math

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact_manual

plotting_backend = "plotly-static"

In [3]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/multivariate/ER_Europe_subset_10nodes.csv"
)
df["ID"] = df["ID"].astype(str)
df["ds"] = pd.to_datetime(df["ds"])
# # use one year for faster training
df = df[df["ID"] =="1"]
df=df.drop(columns=["solar","solar_fcs","ID"])
df.head()

,ds,y
0,2012-01-01 00:00:00,75.6549
1,2012-01-01 01:00:00,70.9958
2,2012-01-01 02:00:00,66.6382
3,2012-01-01 03:00:00,62.9920
4,2012-01-01 04:00:00,61.0699


In [4]:
df.shape

(26304, 2)

Distrubution plots

In [6]:
fig = px.line(df, x=df["ds"], y=df["y"])
fig.show()

In [113]:
quantiles = [0.015, 0.985]

params = {
    "n_lags": 24,
    "n_forecasts": 5,
    "n_changepoints": 20,
    "learning_rate": 0.01,
    "ar_layers": [32, 16, 16, 32],
    "epochs": 40,
    "batch_size": 1024,
    "quantiles": quantiles,
}


m = NeuralProphet(**params)
m.set_plotting_backend(plotting_backend)
set_log_level("ERROR")

In [172]:
df_train, df_test = m.split_df(df, valid_p=0.1, local_split=True)
print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

Train shape: (23673, 2)
Test shape: (2655, 2)


In [115]:
metrics_fit = m.fit(df_train, freq="H", metrics=True)

Training: 0it [00:00, ?it/s]

In [122]:
forecast = m.predict(df_test)

Predicting: 24it [00:00, ?it/s]

In [123]:
#Plotting loss
fig = px.line(metrics_fit, x=metrics_fit["epoch"], y=metrics_fit["Loss"])
fig.show()

In [124]:
# plotting RMSE and MSE over epochs during training
fig = go.Figure()
fig.add_trace(go.Scatter(x=metrics_fit['epoch'], y=metrics_fit['RMSE'], mode='lines', name='RMSE'))
fig.add_trace(go.Scatter(x=metrics_fit['epoch'], y=metrics_fit['MAE'], mode='lines', name='MAE'))
fig.show()

In [125]:
forecast.shape

(2655, 26)

In [126]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_test['ds'], y=df_test['y'], mode='lines', name='Real Data'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat1'], mode='lines', name='yhat1'))

fig.show()

In [127]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print(f'R2 Score : {r2_score(forecast.iloc[25:]["y"],forecast.iloc[25:]["yhat1"])}')
print(f'RMSE : {math.sqrt(mean_squared_error(forecast.iloc[25:]["y"],forecast.iloc[25:]["yhat1"]))}')
print(f'MAE : {mean_absolute_error(forecast.iloc[25:]["y"],forecast.iloc[25:]["yhat1"])}')

R2 Score : 0.971570127272827
RMSE : 2.327730537544704
MAE : 1.7485046358220204


In [180]:
df_test

,ds,y
23649,2014-09-12 09:00:00,99.5443
23650,2014-09-12 10:00:00,101.8352
23651,2014-09-12 11:00:00,103.7559
23652,2014-09-12 12:00:00,103.6548
23653,2014-09-12 13:00:00,101.9893
...,...,...
26299,2014-12-31 19:00:00,102.7619
26300,2014-12-31 20:00:00,99.4563
26301,2014-12-31 21:00:00,92.7608
26302,2014-12-31 22:00:00,85.9727


In [188]:
future = m.make_future_dataframe(df_test, periods=1)
m.predict(future)['yhat1'].iloc[24]

Predicting: 24it [00:00, ?it/s]

77.49892